In [1]:
! wget "https://raw.githubusercontent.com/tatsu-lab/stanford_alpaca/main/seed_tasks.jsonl"

--2024-06-10 21:27:52--  https://raw.githubusercontent.com/tatsu-lab/stanford_alpaca/main/seed_tasks.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 110934 (108K) [text/plain]
Saving to: 'seed_tasks.jsonl'

seed_tasks.jsonl    100%[===================>] 108.33K  --.-KB/s    in 0.02s   

2024-06-10 21:27:52 (6.64 MB/s) - 'seed_tasks.jsonl' saved [110934/110934]



In [2]:
!pip install rouge_score

  Preparing metadata (setup.py) ... - done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=c4c120e45778abf90dfadad7e8c2d76d2ce1808754c6b8d442ce8fd67ebcc4f9
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [3]:
import time
import json
import os
import random
import re
import string
from functools import partial
from multiprocessing import Pool
import transformers
import torch

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import numpy as np
import tqdm
from rouge_score import rouge_scorer
from huggingface_hub import login, logout

# login()

In [4]:
seed_data = open("seed_tasks.jsonl").read().splitlines()
seed_data = [json.loads(d) for d in seed_data]
len(seed_data)

175

In [5]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
token = user_secrets.get_secret("HF_TOKEN")

In [6]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
    token=token
)
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

2024-06-10 21:28:17.880480: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-10 21:28:17.880599: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-10 21:28:18.050377: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
prompt_template = """
You are asked to come up with a set of 20 diverse task instructions. These task instructions will be given to a GPT model and we will evaluate the GPT model for completing the instructions.

Here are the requirements:
1. Try not to repeat the verb for each instruction to maximize diversity.
2. The language used for the instruction also should be diverse. For example, you should combine questions with imperative instructions.
3. The type of instructions should be diverse. The list should include diverse types of tasks like open-ended generation, classification, editing, etc.
4. A GPT language model should be able to complete the instruction. For example, do not ask the assistant to create any visual or audio output. For another example, do not ask the assistant to wake you up at 5pm or set a reminder because it cannot perform any action.
5. The instructions should be in English.
6. The instructions should be 1 to 2 sentences long. Either an imperative sentence or a question is permitted.
7. You should generate an appropriate input to the instruction. The input field should contain a specific example provided for the instruction. It should involve realistic data and should not contain simple placeholders. The input should provide substantial content to make the instruction challenging but should ideally not exceed 100 words.
8. Not all instructions require input. For example, when a instruction asks about some general information, "what is the highest peak in the world", it is not necessary to provide a specific context. In this case, we simply put "<noinput>" in the input field.
9. The output should be an appropriate response to the instruction and the input. Make sure the output is less than 100 words.
10. All instruction-output pairs should be seperated by ### mandatorily.
11. Do not begin response with sentences like "Here are the remaining instructions:"! Generate the first response as it should come after the prompt directly.

List of 20 tasks:
"""

In [8]:
def encode_prompt(prompt_instructions):
    """Encode multiple prompt instructions into a single string."""
#     prompt = prompt_template
    prompt = ""
    for idx, task_dict in enumerate(prompt_instructions):
        (instruction, input, output) = task_dict["instruction"], task_dict["input"], task_dict["output"]
        instruction = re.sub(r"\s+", " ", instruction).strip().rstrip(":")
        input = "<noinput>" if input.lower() == "" else input
        prompt += f"###\n"
        prompt += f"{idx + 1}. Instruction: {instruction}\n"
        prompt += f"{idx + 1}. Input:\n{input}\n"
        prompt += f"{idx + 1}. Output:\n{output}\n"
    prompt += f"###\n"
    prompt += f"{idx + 2}. Instruction:"
    return prompt

In [9]:
def post_process_llama_response(num_prompt_instructions, response):
    if response is None:
        return []
    raw_instructions = response.strip()
    raw_instructions = re.split("###", raw_instructions)
    instructions = []
    for idx, inst in enumerate(raw_instructions):
        inst = inst.strip()
        splitted_data = re.split(f"\d+\.\s+(Instruction|Input|Output):", inst)
        if len(splitted_data) != 7:
            continue
        else:
            inst = splitted_data[2].strip()
            input = splitted_data[4].strip()
            input = "" if input.lower() == "<noinput>" else input
            output = splitted_data[6].strip()
        # filter out too short or too long instructions
        if len(inst.split()) <= 3 or len(inst.split()) > 150:
            continue
        # filter based on keywords that are not suitable for language models.
        blacklist = ["image","images","graph","graphs","picture","pictures","file","files","map","maps","draw","plot","go to","video","audio","music","flowchart","diagram"]
        blacklist += []
        if any(find_word_in_string(word, inst) for word in blacklist):
            continue
        # We found that the model tends to add "write a program" to some existing instructions, which lead to a lot of such instructions.
        # And it's a bit comfusing whether the model need to write a program or directly output the result.
        # Here we filter them out.
        # Note this is not a comprehensive filtering for all programming instructions.
        if inst.startswith("Write a program"):
            continue
        # filter those starting with punctuation
        if inst[0] in string.punctuation:
            continue
        # filter those starting with non-english character
        if not inst[0].isascii():
            continue
        instructions.append({"instruction": inst, "input": input, "output": output})
    return instructions

In [10]:
def find_word_in_string(w, s):
    return re.compile(r"\b({0})\b".format(w), flags=re.IGNORECASE).search(s)

In [11]:
def generate_instruction_following_data(
    output_dir="./",
    seed_tasks_path="./seed_tasks.jsonl",
    num_instructions_to_generate=5,
    num_prompt_instructions=5,
    request_batch_size=1,
    temperature=1.0,
    top_p=1.0,
    num_cpus=1,
):
    seed_tasks = [json.loads(l) for l in open(seed_tasks_path, "r")]
    seed_instruction_data = [
        {"instruction": t["instruction"], "input": t["instances"][0]["input"], "output": t["instances"][0]["output"]}
        for t in seed_tasks
    ]
    print(f"Loaded {len(seed_instruction_data)} human-written seed instructions")

    os.makedirs(output_dir, exist_ok=True)
    request_idx = 0
    # load the LM-generated instructions
    machine_instruction_data = []
    if os.path.exists(os.path.join(output_dir, "regen.json")):
        machine_instruction_data = json.loads(open(os.path.join(output_dir, "regen.json"), 'r').read())
        print(f"Loaded {len(machine_instruction_data)} machine-generated instructions")

    # similarities = {}
    scorer = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=False)

    # now let's generate new instructions!
    progress_bar = tqdm.tqdm(total=num_instructions_to_generate)
#     if machine_instruction_data:
#         progress_bar.update(len(machine_instruction_data))

    # first we tokenize all the seed instructions and generated machine instructions
    all_instructions = [d["instruction"] for d in seed_instruction_data] + [
        d["instruction"] for d in machine_instruction_data
    ]
    all_instruction_tokens = [scorer._tokenizer.tokenize(inst) for inst in all_instructions]
    cur_instructions = 0
    while cur_instructions < num_instructions_to_generate:
        request_idx += 1

        batch_inputs = []
        for _ in range(request_batch_size):
            # only sampling from the seed tasks
            prompt_instructions = random.sample(seed_instruction_data, num_prompt_instructions)
            prompt = encode_prompt(prompt_instructions)
            batch_inputs.append(prompt)

        # changed from openAI's to local              
        decoding_args = {
            'temperature': temperature,
            'max_new_tokens': 3072,  # hard-code to maximize the length. the requests will be automatically adjusted
            'top_p': top_p,
        }
        request_start = time.time()
        
        # changed from openAI         
        results = localModelCompletion(
            prompts=batch_inputs,
            decoding_args=decoding_args
        )
        request_duration = time.time() - request_start

        process_start = time.time()
        instruction_data = []
        for result in results:
            new_instructions = post_process_llama_response(num_prompt_instructions, result)
            instruction_data += new_instructions

        total = len(instruction_data)
        keep = 0
        for instruction_data_entry in instruction_data:
            # computing similarity with the pre-tokenzied instructions
            new_instruction_tokens = scorer._tokenizer.tokenize(instruction_data_entry["instruction"])
            with Pool(num_cpus) as p:
                rouge_scores = p.map(
                    partial(rouge_scorer._score_lcs, new_instruction_tokens),
                    all_instruction_tokens,
                )
            rouge_scores = [score.fmeasure for score in rouge_scores]
            most_similar_instructions = {
                all_instructions[i]: rouge_scores[i] for i in np.argsort(rouge_scores)[-10:][::-1]
            }
            if max(rouge_scores) > 0.7:
                continue
            else:
                keep += 1
            instruction_data_entry["most_similar_instructions"] = most_similar_instructions
            instruction_data_entry["avg_similarity_score"] = float(np.mean(rouge_scores))
            machine_instruction_data.append(instruction_data_entry)
            all_instructions.append(instruction_data_entry["instruction"])
            all_instruction_tokens.append(new_instruction_tokens)
            progress_bar.update(1)
            cur_instructions+=1   
        process_duration = time.time() - process_start
        print(f"Request {request_idx} took {request_duration:.2f}s, processing took {process_duration:.2f}s")
        print(f"Generated {total} instructions, kept {keep} instructions")
        output = json.dumps(machine_instruction_data)
        open(os.path.join(output_dir, "regen.json"), 'w').write(output)

In [12]:
def localModelCompletion(prompts, decoding_args):
    instructions = []
    for prompt in prompts:
        messages = [
            {"role": "system", "content": prompt_template},
            {"role": "user", "content": prompt},
        ]

        terminators = [
            pipeline.tokenizer.eos_token_id,
            pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
        ]
        outputs = pipeline(
            messages,
            eos_token_id=terminators,
            do_sample=True,
            **decoding_args,
            pad_token_id=pipeline.tokenizer.eos_token_id
        )
            
        instruction_txt = outputs[0]["generated_text"][-1]['content']
        instructions.append(instruction_txt)
    return instructions

In [13]:
def pretty_print_instructions(data):
    print("Here are the synthetic instructions generated by the model: \n")
    for d in data:
        print('INSTRUCTION:')
        print(d['instruction'])
        print('INPUT:')
        print(d['input'])
        print('OUTPUT:')
        print(d['output'])
        print('\n------------------------------\n')

In [14]:
#generate 300 instructions
generate_instruction_following_data(
        output_dir="./",
        seed_tasks_path="./seed_tasks.jsonl",
        num_instructions_to_generate=300,
        num_prompt_instructions=8,
        request_batch_size=3,
        temperature=0.8,
        top_p=1.0,
        num_cpus=4,
    )

Loaded 175 human-written seed instructions


  0%|          | 0/300 [00:00<?, ?it/s]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
  4%|▎         | 11/300 [06:18<16:19,  3.39s/it]

Request 1 took 375.61s, processing took 3.37s
Generated 28 instructions, kept 11 instructions


 14%|█▍        | 42/300 [11:15<00:38,  6.75it/s]

Request 2 took 293.00s, processing took 3.95s
Generated 34 instructions, kept 31 instructions


...

Request 19 took 356.39s, processing took 1.25s
Generated 10 instructions, kept 8 instructions


100%|█████████▉| 299/300 [1:42:21<00:00,  2.35it/s]

Request 20 took 364.40s, processing took 4.37s
Generated 35 instructions, kept 20 instructions


301it [1:45:04, 20.95s/it]

Request 21 took 162.55s, processing took 0.79s
Generated 6 instructions, kept 2 instructions


In [15]:
machine_instruction_data = json.loads(open(os.path.join('./', "regen.json"), 'r').read())

In [16]:
len(machine_instruction_data)

301

In [17]:
pretty_print_instructions(machine_instruction_data[:20])

Here are the synthetic instructions generated by the model: 

INSTRUCTION:
Write a short summary of a famous book.
INPUT:
To Kill a Mockingbird by Harper Lee
OUTPUT:
To Kill a Mockingbird is a classic novel set in the Deep South during the 1930s. The story revolves around Scout Finch, a young girl who learns about the injustices of the adult world through her father, Atticus Finch, a lawyer who defends a black man accused of raping a white woman.

------------------------------

INSTRUCTION:
Edit the following essay for grammar, punctuation, and clarity.
INPUT:
The teacher gave the students a lot homework today. They have to finish all the assignment by tomorrow. The class is very busy and no one like it. The student are very tired after the class is finish. The teacher are very nice and give us alot of help.
OUTPUT:
The teacher assigned a significant amount of homework today. Students are expected to complete all assignments by tomorrow. The class is overwhelmed with work and no one e